In [1]:
import ast
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
with open('results/pytorch_rnn_prequential_summed_2022-04-19.txt', 'r') as file:
    text_pred = ast.literal_eval(file.read())

In [3]:
with open('results/nn_spatialinfo_prequential_summed_2022-04-14.txt', 'r') as file:
    spatial_info = ast.literal_eval(file.read())

In [4]:
model_results = pd.read_csv('../../opportunistic_planning/all_lowest_error/results/results_mean_2D_n100_2022-04-19.csv')

In [5]:
model_results = model_results.T
new_header = model_results.iloc[0]
model_results = model_results[1:]
model_results.columns = new_header

In [6]:
lowest_mean = min(model_results.loc['mean'])
lowest_mean_idx = model_results.columns[(model_results.loc['mean'] == lowest_mean)]

In [7]:
lowest_mean_idx

Index(['c: 1.5; k: 0.2,0.3,1.3; xy'], dtype='object', name='Unnamed: 0')

In [8]:
results = model_results[lowest_mean_idx[0]][:-2].values

In [9]:
len(results) == len(text_pred) == len(spatial_info)

True

In [10]:
df = pd.read_csv('../../opportunistic_planning/all_lowest_error/all_task_environments_list_2022-04-01.csv', header=0)

In [26]:
prediction_errors = pd.DataFrame()
prediction_errors['ID'] = model_results['ID'][:-2]
prediction_errors['text_pred'] = text_pred
prediction_errors['spatial_info'] = spatial_info
prediction_errors['model'] = [float(x) for x in results]
prediction_errors['sequence'] = list(df['sequence'])

In [88]:
#prediction_errors['model'].value_counts(sort=True)

In [83]:
value_counts = pd.DataFrame(prediction_errors['model'].value_counts(sort=True))
value_counts['spatial_info'] = prediction_errors['spatial_info'].value_counts()
value_counts['text_pred'] = prediction_errors['text_pred'].value_counts()

value_counts.index.name = 'error'
value_counts = value_counts.sort_values('error', axis=0)
value_counts.reset_index(inplace=True)
#value_counts

In [82]:
#value_counts.to_csv('results/value_counts.csv', header=True, index=False)

In [145]:
%matplotlib qt

# frequencies in subplots

fig = plt.figure(facecolor="w")

ax1 = plt.subplot(131)
prediction_errors['spatial_info'].value_counts().sort_values().plot(kind = 'barh',
                                                                    color='dodgerblue',
                                                                    alpha=0.9)
ax1.set_xlabel('NN with spatial info', fontsize=16)

ax2 = plt.subplot(132)
prediction_errors['text_pred'].value_counts().sort_values().plot(kind = 'barh',
                                                                 color='darkgreen', 
                                                                 alpha=0.9)
ax2.set_xlabel('NN text prediction', fontsize=16)

ax3 = plt.subplot(133)
prediction_errors['model'].value_counts().sort_values().plot(kind = 'barh',
                                                             color='darkred', 
                                                             alpha=0.9)
ax3.set_xlabel('model', fontsize=16)

ax1.set_ylabel('accumulated prediction error', fontsize=18)
plt.suptitle('error frequencies', fontsize=18)
plt.show()

In [108]:
# hist bar chart
%matplotlib qt

fig = plt.figure(facecolor="w")
#value_counts['spatial_info'].plot(kind = 'bar', color='dodgerblue', alpha=0.9,)
#value_counts['text_pred'].plot(kind = 'bar', color='darkgreen', alpha=0.9)
#value_counts['model'].plot(kind = 'bar', color='darkred', alpha=0.9)

value_counts.plot(x="error", y=['spatial_info', 'text_pred', 'model'], kind='bar')
plt.ylabel('number of occurrences', fontsize=18)
plt.xlabel('error', fontsize=18)

ax = value_counts.plot.bar()

for container in ax.containers:
    ax.bar_label(container)

plt.legend(fontsize=14)
plt.show()

In [101]:
# separate bar charts
%matplotlib qt

fig = plt.figure(facecolor="w", figsize=(16,12))
prediction_errors.plot(x="ID", y=['spatial_info', 'text_pred', 'model'], kind='bar')
plt.show()

In [54]:
prediction_errors.describe().round(2)

,text_pred,spatial_info,model
count,314.00,314.00,314.00
mean,4.47,2.14,3.48
std,2.20,1.69,2.40
min,1.00,0.00,0.00
25%,3.00,1.00,2.00
50%,5.00,2.00,3.00
75%,6.00,3.00,5.00
max,14.00,11.00,13.00
